In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
from collections import OrderedDict
from matplotlib.backends.backend_pdf import PdfPages
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from adapters import *
from dataset import NumpyCsvDataSet
from runner import ExptRunner
from networks.autoencoder import AutoEncoder
from networks.composedautoencoder import ComposedAutoEncoder
from networks.DenseAutoEncoder import DenseAutoEncoder
from networks.ConvVae import ConvVae
from networks.vae import VAE
from networks.etpr.autoencoder import autoencoder as etprautoenc
from networks.dense import Dense
from networks.dense import DenseForPolicy
from networks.imageencoder import ImageEncoder
from networks.multinet import *
from networks.lossfunctions import *
from helpers import ReportResults
from policyeval import PolicyEvaluator
from policyeval import eval_policy_accuracy
import utils

In [3]:
import warnings
warnings.filterwarnings("ignore", category=torch.serialization.SourceChangeWarning)

In [4]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

In [5]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [8]:
start = time.time()
data_folder = 'data/demoplanner/version1'
# v1_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v1_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 2.3487699031829834 sec


In [6]:
start = time.time()
data_folder = 'data/demoplanner/version2'
# v2_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v2_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 11.170909881591797 sec


In [7]:
test_data = v2_test_data.data
# train_data = v2_train_data.data

In [8]:
# print (train_data.shape)
# print (train_data.device)

print (test_data.shape)
print (test_data.device)

torch.Size([6648, 2058])
cpu


In [9]:
t_cpu = test_data.data.cpu()
start_indices = []
index = 0
while index < len(t_cpu):
    start_indices.append(index)
    start, goal = t_cpu[index, :2], t_cpu[index, 2:4]
    index += int(max(np.abs(goal - start)))
print (start_indices)
print (len(start_indices))

[0, 22, 31, 34, 43, 50, 70, 86, 90, 106, 114, 122, 138, 152, 157, 173, 184, 191, 211, 219, 224, 226, 249, 260, 269, 279, 290, 307, 324, 340, 355, 367, 376, 400, 412, 426, 436, 453, 474, 485, 489, 509, 530, 536, 545, 554, 579, 597, 608, 621, 643, 651, 656, 664, 676, 683, 692, 706, 713, 740, 759, 774, 792, 802, 819, 841, 853, 878, 898, 905, 914, 925, 934, 944, 954, 974, 984, 998, 1007, 1008, 1015, 1025, 1043, 1061, 1065, 1089, 1094, 1103, 1114, 1126, 1129, 1152, 1160, 1177, 1183, 1196, 1207, 1232, 1243, 1265, 1274, 1293, 1302, 1316, 1328, 1350, 1358, 1371, 1375, 1397, 1402, 1422, 1439, 1442, 1454, 1461, 1481, 1500, 1516, 1536, 1550, 1559, 1565, 1577, 1601, 1626, 1645, 1669, 1681, 1694, 1702, 1720, 1729, 1738, 1753, 1777, 1792, 1793, 1794, 1802, 1814, 1818, 1836, 1842, 1865, 1887, 1909, 1915, 1925, 1931, 1941, 1963, 1979, 1998, 2023, 2042, 2068, 2093, 2096, 2106, 2124, 2136, 2149, 2160, 2180, 2194, 2206, 2213, 2236, 2242, 2248, 2260, 2269, 2291, 2304, 2312, 2322, 2336, 2350, 2360, 2380, 2

In [10]:
def get_trajectory(start_indices, traj_index):
    rstart = start_indices[traj_index]
    rend = start_indices[traj_index+1] if traj_index < len(start_indices)-1 else len(t_cpu)
    return t_cpu[rstart:rend, :]
    
def get_trajectories(start_indices):
    for i in range(len(start_indices)):
        yield i, get_trajectory(start_indices, i)

In [11]:
def load_pretrained(folder):
    chkptFile = 'runs/pretrained/' + folder + '/train_checkpoint.tar'
    return torch.load(chkptFile, map_location=device)['model']

pretrain_policy = Dense([4, 16, 16, 6], use_last_act=False, prefix='enc').to(device)
pretrain_policy.load_state_dict(torch.load('runs/pretrained/04-08-10-24-50-Policy-V2-Dense-CE/autoenc.pth', 
                                  map_location=device))

pretrain_dynamics = Dense([4, 16, 2], prefix='enc').to(device)
pretrain_dynamics.load_state_dict(torch.load('runs/pretrained/03-31-18-02-52-Dynamics-V1-Dense-SmoothL1/autoenc.pth', 
                                    map_location=device))

pretrain_imgEncDec = ComposedAutoEncoder(layers_channels=[16,16,16,16], useMaxPool=True, device=device).to(device)
pretrain_imgEncDec.load_state_dict(torch.load('runs/pretrained/03-28-14-10-20-DemoPl-V2-ComposedAutoEncoderWithMaxPool-SmoothL1-/autoenc.pth',
                               map_location=device))

pretrain_envEncDec = load_pretrained('04-20-11-03-59-EnvAutoEnc-V2-DenseAutoEncoder-MSE')
pretrain_ImgToEnvBaseLine = load_pretrained('04-21-12-51-48-ImageToEnvFunctionBaseLine-V2-3Conv2Dense-MSE')
latentModel0422 = load_pretrained('04-22-23-59-35-TriLossOnCombinedNet-V2-ImageEnvEncoder-MSE_Adam')
latentModel0427 = load_pretrained('04-27-21-54-52-QuadLossOnCombinedNet-V2-MultiNet-MSE')

In [12]:
adjusted_ImgToEnv_BaseLine = nn.Sequential(OrderedDict([
    ('Conv', ImageEncoder(1, [16,16,16], 'imgenc', useMaxPool=True, addFlatten=True)),
    ('Dense', Dense([256, 4]))
]))
adjusted_ImgToEnv_BaseLine.load_state_dict(pretrain_ImgToEnvBaseLine.state_dict())

<All keys matched successfully>

In [13]:
def latentFilter_gt_x_latent_y(data, latent_output):
    return torch.cat((Xt_scaled_adapter(data), latent_output[:, 2:4]), dim=1)

In [14]:
chkpt = torch.load('runs/06-16/06-16-20-33-57-FixImagEncAndPolicyTrainD3EnvDec-V2-LatentPolicyNet-CE/train_checkpoint.tar', map_location=device)

In [15]:
net = chkpt['model']

In [16]:
net.latentFilter

<function __main__.latentFilter_gt_x_latent_y(data, latent_output)>

In [17]:
net

LatentPolicyNet(
  (latent): Sequential(
    (EncNet): ImageEncoderFlatInput(
      (net): Sequential(
        (imgenc_conv0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (imgenc_relu0): ReLU()
        (imgenc_maxpool0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (imgenc_conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (imgenc_relu1): ReLU()
        (imgenc_maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (imgenc_conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (imgenc_relu2): ReLU()
        (imgenc_maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (imgenc_conv3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (imgenc_relu3): ReLU()
        (imgenc_maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      

In [20]:
persist_to_disk = True

def label_adapter_end_to_end_net(data):
    # ut, xt+, It+, yt
    return torch.cat((data[:, 1028:1032], data[:, 1034:], data[:, 2:4]), dim=1)

def label_adapter_latent_policy_net(data):
    # ut, xt+, It+
    return torch.cat((data[:, 1028:1032], data[:, 1034:]), dim=1)

def run_eval_end_to_end_net(chkpt_file):
    log_folder = os.path.dirname(chkpt_file)
    chkpt = torch.load(log_folder + '/train_checkpoint.tar', map_location=device)
    net = chkpt['model']
    label_adapter = label_adapter_latent_policy_net
    
    errors = torch.zeros((len(start_indices), 6))
    allLabels = []
    allPredictions = []
    for index, trajectory in get_trajectories(start_indices):
        labels = label_adapter(trajectory)
        with torch.no_grad():
            predictions = net.rollout(trajectory)
        allLabels.append(labels)
        allPredictions.append(predictions)
        errors[index][0] = index
        errors[index][1] = len(trajectory)
        errors[index][2] = eval_policy_accuracy(predictions[:, 0:2], labels[:, 0:2])
        errors[index][3] = F.l1_loss(predictions[:, 0:2], labels[:, 0:2])
        errors[index][4] = F.l1_loss(predictions[:, 2:4], labels[:, 2:4])
        errors[index][5] = F.l1_loss(predictions[-1, 2:4], labels[-1, 2:4])
    
    aggregate_row_headers = ['Sum', 'Average', 'Min Index', 'Min Value', 'Max Index', 'Max Value', 'Counts']
    aggregates = torch.zeros((7, 6))
    aggregates[0] = torch.sum(errors, dim=0)
    aggregates[1] = aggregates[0] / len(start_indices)
    aggregates[3], aggregates[2] = torch.min(errors, dim=0)
    aggregates[5], aggregates[4] = torch.max(errors, dim=0)
    lower_threshholds = torch.Tensor([-1, 10, 99]) # Total trajectories, trajectories of len > 10, of accuracy > 99
    upper_threshholds = torch.Tensor([0.01, 0.01, 0.01]) # trajectories of latent loss, dynamics step loss, goal loss < 0.01
    aggregates[6] = torch.sum(torch.cat((
        torch.gt(errors[:, :3], lower_threshholds),
        torch.lt(errors[:, 3:], upper_threshholds)), dim=1), dim=0)
    agg = aggregates.numpy()
    
    result_folder = log_folder + '/Evaluations'
    if persist_to_disk:
        os.makedirs(result_folder)

        with open(result_folder + '/trajectory_eval_metrics.csv', 'a') as f:
            f.write('Trajectory Index,Trajectory Length,Policy Accuracy,Policy L1 Loss,Dynamics Loss (step),Goal Deviation\n')
            f.write('Aggregates:\n')
            for i in range(len(aggregate_row_headers)):
                f.write(aggregate_row_headers[i])
                f.write(',')
                f.write(str.join(',', [str(val) for val in agg[i, 1:]]))
                f.write('\n')
            f.write('\nInidividual:\n')
            np.savetxt(f, errors.numpy(), fmt='%5.2f', delimiter=',')
    
    sample_descriptions = ['Best Policy Accuracy','Least Policy L1 Error',
                           'Least Trajectory Error','Least Goal Error',
                           'Worst Policy Accuracy','Highest Policy L1 Error',
                           'Highest Trajectory Error','Highest Goal Error']
    # For policy, min means worst, for other errors, min means best
    indices = torch.cat((aggregates[4, 2:3], aggregates[2, 3:], aggregates[2, 2:3], aggregates[4, 3:])).long()
    values = torch.cat((aggregates[5, 2:3], aggregates[3, 3:], aggregates[3, 2:3], aggregates[5, 3:]))
    print ('-------------------------------------------------------------')
    print (log_folder)
    print ('-------------------------------------------------------------')
    print ('Number of Trajectories Ended up Within Goal Region: {}'.format(aggregates[6, 5].item()))
    print ('Average Policy Accuracy: {}'.format(aggregates[1, 2].item()))
    print ('Average Latent Loss: {}'.format(aggregates[1, 3].item()))
    print ('Average Trajectory Loss: {}'.format(aggregates[1, 4].item()))
    print ('Average Goal Loss: {}'.format(aggregates[1, 5].item()))
    print ('')
    for i in range(len(sample_descriptions)):
        print ('{}: Index = {}, Value = {}'.format(sample_descriptions[i], indices[i], values[i]))
        if i == 3:
            print ('')

    if persist_to_disk:
        for i in range(len(sample_descriptions)):
            index = indices[i]
            labels = allLabels[index]
            predictions = allPredictions[index]
            filename = '/traj_' + str(index.item()) + '_' + str.join('_', str.split(sample_descriptions[i])) + '.pdf'
            with PdfPages(result_folder + filename) as pdf:
                for i in range(len(labels)):
                    fig = plt.figure()
                    plt.subplot(1,2,1)
                    plt.imshow(labels[i, 4:].reshape(32,32), cmap=plt.get_cmap("gray"))
                    plt.subplot(1,2,2)
                    plt.imshow(predictions[i, 4:].reshape(32,32), cmap=plt.get_cmap("gray"))
                    pdf.savefig(fig)
                    plt.close(fig)

In [21]:
start = time.time()

for chkpt_file in utils.enumerate_files(rootdir='runs/06-16'):
    run_eval_end_to_end_net(chkpt_file)
    # shutil.rmtree(os.path.dirname(chkpt_file) + '/Evaluations', ignore_errors=True)
    # os.rename(os.path.dirname(chkpt_file) + '/Evaluations', os.path.dirname(chkpt_file) + '/Old_Evaluations')

print ("Time Taken: {} sec".format(time.time() - start))

-------------------------------------------------------------
runs/06-16/06-16-20-33-57-FixImagEncAndPolicyTrainD3EnvDec-V2-LatentPolicyNet-CE
-------------------------------------------------------------
Number of Trajectories Ended up Within Goal Region: 89.0
Average Policy Accuracy: 62.477874755859375
Average Latent Loss: 0.28515496850013733
Average Trajectory Loss: 1.2255933284759521
Average Goal Loss: 3.693000078201294

Best Policy Accuracy: Index = 494, Value = 100.0
Least Policy L1 Error: Index = 494, Value = 0.0
Least Trajectory Error: Index = 494, Value = 0.0
Least Goal Error: Index = 494, Value = 0.0

Worst Policy Accuracy: Index = 489, Value = 0.0
Highest Policy L1 Error: Index = 153, Value = 1.2999999523162842
Highest Trajectory Error: Index = 153, Value = 12.279999732971191
Highest Goal Error: Index = 200, Value = 28.0
-------------------------------------------------------------
runs/06-16/06-16-23-26-42-FixImagEncAndPolicyTrainD3EnvDec-V2-LatentPolicyNet-CE
-------------

In [ ]:
log_folder = 'runs/05-05/run3/05-05-14-29-58-TrainPolicyOnlyInEndToEnd-V2-EndToEndEnv-CE'
chkpt = torch.load(log_folder + '/train_checkpoint.tar', map_location=device)
net = chkpt['model']
label_adapter = lambda data: torch.cat((data[:, 2:4], data[:, 1028:1032], data[:, 1034:]), dim=1)
e = PolicyEvaluator(net, distance_func=F.l1_loss, label_adapter=label_adapter)

In [88]:
errors = torch.zeros((len(start_indices), 5))
allLabels = []
allPredictions = []
for index, trajectory in get_trajectories(start_indices):
    goal_err, step_err, labels, predictions = e.eval_single_trajectory(trajectory)
    allLabels.append(labels)
    allPredictions.append(predictions)
    errors[index][0] = index
    errors[index][1] = len(trajectory)
    errors[index][2] = F.l1_loss(predictions[:, 0:2], labels[:, 0:2])
    errors[index][3] = eval_policy_accuracy(predictions[:, 2:4], labels[:, 2:4])
    errors[index][4] = F.l1_loss(predictions[:, 4:6], labels[:, 4:6])


In [89]:
indices = [torch.argmin(errors[:, 2]).item(), torch.argmax(errors[:, 2]).item(),
           torch.argmin(errors[:, 3]).item(), torch.argmax(errors[:, 3]).item()]
print (indices)
print (errors[indices])

[159, 344, 85, 499]
tensor([[1.5900e+02, 1.8000e+01, 2.1521e-02, 1.0000e+02, 0.0000e+00],
        [3.4400e+02, 1.0000e+01, 1.3117e+00, 9.0000e+01, 5.0000e-02],
        [8.5000e+01, 5.0000e+00, 1.7077e-01, 6.0000e+01, 2.0000e-01],
        [4.9900e+02, 1.3000e+01, 8.3551e-02, 1.0000e+02, 0.0000e+00]])


In [90]:
index = 85
print (index)

85


In [91]:
traj = get_trajectory(start_indices, index)
print (len(traj))

5


In [92]:
print (traj[:, :6])

tensor([[ 8., 24., 13., 26.,  0.,  0.],
        [ 9., 24., 13., 26.,  0.,  0.],
        [10., 24., 13., 26.,  0.,  0.],
        [11., 24., 13., 26.,  0.,  0.],
        [12., 25., 13., 26.,  0.,  0.]])


In [93]:
goal_err, step_err, labels, predictions = e.eval_single_trajectory(traj)
print (len(traj))
print (goal_err)
print (step_err)

5
tensor(0.6183)
tensor(0.9421)


In [94]:
print (torch.cat((labels[:, :2], predictions[:, :2]), dim=1))
print (torch.cat((labels[:, 2:6], predictions[:, 2:6]), dim=1))

tensor([[13.0000, 26.0000, 12.7572, 26.0149],
        [13.0000, 26.0000, 13.0115, 26.0237],
        [13.0000, 26.0000, 12.9172, 26.1644],
        [13.0000, 26.0000, 13.2280, 25.9498],
        [13.0000, 26.0000, 13.6645, 25.7751]])
tensor([[ 1.,  0.,  9., 24.,  1.,  0.,  9., 24.],
        [ 1.,  0., 10., 24.,  1.,  0., 10., 24.],
        [ 1.,  0., 11., 24.,  1.,  1., 11., 25.],
        [ 1.,  1., 12., 25.,  1.,  1., 12., 26.],
        [ 1.,  1., 13., 26.,  1.,  0., 13., 26.]])


In [57]:
indices = [66, 483, 13]

In [ ]:
for index in indices:
    print ('-------------------------------------------------------------')
    print ('                         {}'.format(index))
    print ('-------------------------------------------------------------')
#     folder = log_folder + '/Evaluations/{}/'.format(index)
#     os.makedirs(folder)
    labels = allLabels[index]
    predictions = allPredictions[index]
    for i in range(len(labels)):    
        fig = plt.figure()
        plt.subplot(1,2,1)
        plt.imshow(labels[i, 6:].reshape(32,32), cmap=plt.get_cmap("gray"))
        plt.subplot(1,2,2)
        plt.imshow(predictions[i, 6:].reshape(32,32), cmap=plt.get_cmap("gray"))
#         fig.savefig(folder + '/{}.png'.format(i))
        plt.show()


In [ ]:
print ('Trajectory with highest goal error = {}'.format(torch.argmax(errors[:, 1])))
print ('Trajectory with highest average step error = {}'.format(torch.argmax(errors[:, 2])))
print ('Trajectory Length, Goal Error, Average Error')
for i in range(len(errors)):
    print('{},{},{}'.format(*errors[i]))

In [ ]:
traj_no = 386 
rstart = start_indices[traj_no]
rend = start_indices[traj_no+1] if traj_no < len(start_indices) else len(test_data)
trajectory = t_cpu[rstart:rend, :]
f = torch.cat((trajectory[:, :4], trajectory[:, 1028:1030]), dim=1)
print (len(f))
print (f)

In [ ]:
goal_err, step_err, pred_traj, orig_traj, policy_pred, dyn_pred = e.eval_single_trajectory(trajectory)

print ('Last step error = {}'.format(goal_err))
print ('Average error over all steps = {}'.format(step_err))
print ('Original Trajectory and Predicted Trajectory:')
print (torch.cat((orig_traj, pred_traj), dim=1))
print ('Stepwise deviation from original trajectory:')
print (torch.abs(orig_traj-pred_traj))
print ('Prediction from Policy:')
print (policy_pred - 1.0) # Using minus to bring the policy output to the values we expect
print ('Prediction from Dynamics:')
print (dyn_pred * 2.0 - 1.0)  # Again bringing the dynamics prediction to the expected range

In [ ]:
np.set_printoptions(**opt)